# 3D Spheroid Visualization and Animation

This cell generates 3D visualizations and animated GIFs of cell spheroids from simulation data. It loads voxel and state information for each simulation step, assigns colors to unique cell states, creates smooth meshes for each cell, and renders frames using PyVista. The frames are then compiled into an animated GIF to visualize the temporal evolution of the spheroid structure.

In [12]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes
import pyvista as pv
import imageio
import os

GRID_SIZE = (70, 70, 70)
steps = range(1, 201)  # step_00001.jld2 to step_00010.jld2
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# --- Setup colormap once ---
state_colors = {}
unique_states_all = set()

# --- First pass to get all unique states ---
for step in steps:
    with h5py.File(f"sim_output/7/step_{step:05d}.jld2", "r") as f:
        cell_states = {
            int(k): f["cell_states"][k][()].decode("utf-8")
            for k in f["cell_states"]
        }
        unique_states_all.update(cell_states.values())

cmap = plt.get_cmap("tab10")
unique_states = sorted(unique_states_all)
state_colors = {
    state: cmap(i / len(unique_states))[:3]
    for i, state in enumerate(unique_states)
}

# --- Visualization loop ---
for step in steps:
    with h5py.File(f"sim_output/7/step_{step:05d}.jld2", "r") as f:
        cell_voxels = {
            int(k): np.array([list(t) for t in f["cell_voxels"][k][()]]).astype(int)
            for k in f["cell_voxels"]
        }
        cell_states = {
            int(k): f["cell_states"][k][()].decode("utf-8")
            for k in f["cell_states"]
        }

    plotter = pv.Plotter(off_screen=True)
    plotter.set_background("white")

    def voxels_to_mesh(voxels, grid_size):
        volume = np.zeros(grid_size, dtype=np.uint8)
        for x, y, z in voxels:
            if 0 <= x < grid_size[0] and 0 <= y < grid_size[1] and 0 <= z < grid_size[2]:
                volume[x, y, z] = 1
        try:
            verts, faces, _, _ = marching_cubes(volume, level=0.5)
            faces_vtk = np.hstack([[3] + list(face) for face in faces]).astype(np.uint32).reshape(-1, 4)
            return pv.PolyData(verts, faces_vtk)
        except Exception:
            return None

    for cid, voxels in cell_voxels.items():
        mesh = voxels_to_mesh(voxels, GRID_SIZE)
        if mesh is not None:
            color = state_colors[cell_states[cid]]
            plotter.add_mesh(mesh, color=color, opacity=1.0, show_edges=False)

    # Bounding box
    bbox = pv.Cube(center=(GRID_SIZE[0]/2, GRID_SIZE[1]/2, GRID_SIZE[2]/2),
                   x_length=GRID_SIZE[0], y_length=GRID_SIZE[1], z_length=GRID_SIZE[2])
    plotter.add_mesh(bbox, color="black", style="wireframe", opacity=0.1, line_width=1)

    # Axes, title, legend
    plotter.show_bounds(grid='back', location='outer', all_edges=False, ticks='outside',
                        xlabel='X', ylabel='Y', zlabel='Z', font_size=14)
    plotter.view_vector((1, 1, 0.5))
    plotter.add_title(f"Cells at step {step}", font_size=20)

    legend_entries = [(state, state_colors[state]) for state in unique_states]
    plotter.add_legend(legend_entries, bcolor='white', border=False, size=(0.15, 0.15), loc='upper right')

    frame_path = os.path.join(output_dir, f"frame_{step:03d}.png")
    plotter.screenshot(frame_path)
    plotter.close()

# --- Create video ---
images = [imageio.imread(os.path.join(output_dir, f"frame_{step:03d}.png")) for step in steps]
imageio.mimsave("simulation_4.gif", images, fps=2)  # You can also use .mp4 if preferred


C:\Users\Standard\AppData\Local\Temp\ipykernel_50768\320512414.py:86: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(os.path.join(output_dir, f"frame_{step:03d}.png")) for step in steps]


# Rotating 3D Spheroid Visualization

This cell loads simulation data for a single time step, generates smooth 3D meshes for each cell, and creates a rotating animation of the spheroid structure. The animation provides a comprehensive view of the spatial organization and cell state distribution within the spheroid by rotating the camera around the model and saving the frames as a GIF.

In [13]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes
import pyvista as pv
import os
import imageio

# --- Simulation data file ---
STEP = 200
path = f"sim_output/7/step_{STEP:05d}.jld2"

# --- Load simulation data ---
with h5py.File(path, "r") as f:
    step = f["step"][()]
    cell_voxels = {
        int(k): np.array([list(t) for t in f["cell_voxels"][k][()]]).astype(int)
        for k in f["cell_voxels"]
    }
    cell_states = {
        int(k): f["cell_states"][k][()].decode("utf-8")
        for k in f["cell_states"]
    }

GRID_SIZE = (80, 80, 80)

# --- Prepare colormap ---
unique_states = sorted(set(cell_states.values()))
cmap = plt.get_cmap("tab10")
state_colors = {
    state: cmap(i / len(unique_states))[:3]
    for i, state in enumerate(unique_states)
}

# --- PyVista setup ---
plotter = pv.Plotter(off_screen=True)
plotter.set_background("white")

# --- Helper: Convert voxel list to mesh ---
def voxels_to_mesh(voxels, grid_size):
    volume = np.zeros(grid_size, dtype=np.uint8)
    for x, y, z in voxels:
        if 0 <= x < grid_size[0] and 0 <= y < grid_size[1] and 0 <= z < grid_size[2]:
            volume[x, y, z] = 1
    try:
        verts, faces, _, _ = marching_cubes(volume, level=0.5)
        faces_vtk = np.hstack([[3] + list(face) for face in faces]).astype(np.uint32).reshape(-1, 4)
        mesh = pv.PolyData(verts, faces_vtk)
        return mesh
    except Exception:
        return None

# --- Add cell meshes ---
for cid, voxels in cell_voxels.items():
    mesh = voxels_to_mesh(voxels, GRID_SIZE)
    if mesh is not None:
        color = state_colors[cell_states[cid]]
        plotter.add_mesh(mesh, color=color, opacity=1.0, show_edges=False)

# --- Add bounding box ---
bbox = pv.Cube(center=(GRID_SIZE[0]/2, GRID_SIZE[1]/2, GRID_SIZE[2]/2),
               x_length=GRID_SIZE[0], y_length=GRID_SIZE[1], z_length=GRID_SIZE[2])
plotter.add_mesh(bbox, color="black", style="wireframe", opacity=0.1, line_width=1)

# --- Add legend ---
legend_entries = [(state, state_colors[state]) for state in unique_states]
plotter.add_legend(legend_entries, bcolor='white', border=False, size=(0.15, 0.15), loc='upper right')

# --- Add axis labels and title ---
plotter.show_bounds(grid='back', location='outer', all_edges=False, ticks='outside',
                    xlabel='X', ylabel='Y', zlabel='Z', font_size=14)
plotter.add_title(f"Rotating view of step {step}", font_size=20)

# --- Rotating camera animation ---
n_frames = 60
output_dir = "rotating_frames"
os.makedirs(output_dir, exist_ok=True)

plotter.view_vector((1, 1, 0.5))  # initial view

for i in range(n_frames):
    plotter.camera.Azimuth(360 / n_frames)
    plotter.render()
    frame_path = os.path.join(output_dir, f"rot_{i:03d}.png")
    plotter.screenshot(frame_path)

plotter.close()

# --- Create GIF animation ---
images = [imageio.imread(os.path.join(output_dir, f"rot_{i:03d}.png")) for i in range(n_frames)]
imageio.mimsave("rotating_simulation_3.gif", images, fps=12)
print("✅ Done! Saved rotating_simulation.gif")


C:\Users\Standard\AppData\Local\Temp\ipykernel_50768\1511714809.py:90: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(os.path.join(output_dir, f"rot_{i:03d}.png")) for i in range(n_frames)]


✅ Done! Saved rotating_simulation.gif


# Fast Clean Cut Spheroid Visualization

This cell defines a function to quickly render a clean "cut" view of a 3D spheroid simulation. It loads voxel and state data, assigns colors to cell states, and visualizes only the portion of the spheroid on one side of a specified axis cut. The function supports saving the rendered image and includes a scale bar for spatial reference.

In [13]:
def render_fast_clean_cut(filepath, cut_axis='x', cut_fraction=0.5, grid_shape=(70, 70, 70),
                          state_color_map=None, show=True, save_path=None):
    import h5py
    import numpy as np
    import pyvista as pv
    from matplotlib.colors import to_rgb

    # Load voxel data
    with h5py.File(filepath, "r") as f:
        cell_voxels = {}
        for k in f["cell_voxels"]:
            raw = f["cell_voxels"][k][()]
            voxels = np.stack([raw[name] for name in raw.dtype.names], axis=1) if raw.dtype.names else raw
            cell_voxels[int(k)] = voxels.astype(int)
        cell_states = {int(k): f["cell_states"][k][()].decode("utf-8") for k in f["cell_states"]}

    if state_color_map is None:
        state_color_map = {
            "gbm_p": "#e41a1c", "gbm_q": "#4daf4a", "gbm_n": "#377eb8",
            "gsc_p": "#ff7f00", "gsc_q": "#984ea3", "msc": "#a65628",
        }

    # ID map
    unique_states = sorted(set(cell_states.values()))
    state_to_id = {state: i + 1 for i, state in enumerate(unique_states)}

    # Cut
    axis_idx = {'x': 0, 'y': 1, 'z': 2}[cut_axis]
    cut_threshold = int(grid_shape[axis_idx] * cut_fraction)

    # Fill labeled voxel grid
    labeled = np.zeros(grid_shape, dtype=np.uint8)
    for cid, voxels in cell_voxels.items():
        state = cell_states[cid]
        label = state_to_id[state]
        if voxels[:, axis_idx].mean() <= cut_threshold:
            for x, y, z in voxels:
                if 0 <= x < grid_shape[0] and 0 <= y < grid_shape[1] and 0 <= z < grid_shape[2]:
                    labeled[x, y, z] = label

    # Render
    plotter = pv.Plotter(off_screen=not show, window_size=(800, 800))
    for state, sid in state_to_id.items():
        binary = (labeled == sid).astype(np.uint8)
        if binary.sum() == 0: continue
        color = to_rgb(state_color_map.get(state, "#999999"))
        mesh = pv.wrap(binary).contour(isosurfaces=[0.5])
        plotter.add_mesh(mesh, color=color, opacity=1.0, show_scalar_bar=False)

    plotter.set_background("white")
    plotter.view_isometric()
    plotter.camera.zoom(1.4)
    add_full_x_scalebar(
        plotter,
        grid_shape=grid_shape,
        voxel_size_um=10,
        tick_step_um=350,
        bar_y=-5,
        bar_z=-5
    )




    if save_path:
        plotter.screenshot(save_path)
    if show:
        plotter.show()
def add_full_x_scalebar(plotter, grid_shape=(70, 70, 70), voxel_size_um=10,
                        tick_step_um=100, bar_z=-5, bar_y=-5, label_offset=-3):
    """
    Draws a full-length X-axis scale bar with labeled ticks under the spheroid (in front view).
    - bar_z and bar_y push it visibly in front/below
    """
    import numpy as np

    total_voxels = grid_shape[0]
    total_um = total_voxels * voxel_size_um

    # Start and end X in voxels
    x0 = 0
    x1 = total_voxels

    # Main scale bar line
    line = np.array([[x0, bar_y, bar_z], [x1, bar_y, bar_z]])
    plotter.add_lines(line, color='black', width=2)

    # Tick positions in μm
    ticks_um = np.arange(0, total_um + 1, tick_step_um)
    for um in ticks_um:
        x = um / voxel_size_um
        # Short vertical tick
        tick_line = np.array([[x, bar_y, bar_z], [x, bar_y, bar_z + 0.7]])
        plotter.add_lines(tick_line, color='black', width=1)
        # Label slightly below
        plotter.add_point_labels([[x, bar_y, bar_z + label_offset]], [f"{int(um)} μm"],
                                 font_size=20, text_color='black', shape_opacity=0.0)





# Standalone Legend Generation for Spheroid Cell States

This cell provides a function to generate a standalone legend image for spheroid visualizations. The function takes a mapping of cell states to colors and optionally custom labels, then creates and saves a PNG image of the legend. This is useful for including consistent, publication-ready legends alongside 3D or 2D spheroid figures.

In [32]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

def generate_spheroid_legend(state_color_map, state_labels=None, save_path="legend.png", dpi=300):
    """
    Creates a standalone legend image for spheroid cell states.

    Parameters:
    - state_color_map (dict): state -> color (hex or RGB)
    - state_labels (dict): optional dict to rename legend labels (e.g., 'gbm_p' -> 'Proliferative GBM')
    - save_path (str): output path for the PNG
    - dpi (int): resolution
    """
    if state_labels is None:
        state_labels = {s: s for s in state_color_map}

    legend_elements = [
        Patch(facecolor=state_color_map[state], edgecolor='black', label=state_labels.get(state, state))
        for state in state_color_map
    ]

    fig, ax = plt.subplots(figsize=(4, 0.5 + 0.4 * len(legend_elements)))
    ax.axis('off')
    ax.legend(handles=legend_elements, loc='center', frameon=False, fontsize=12, ncol=1)
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi, bbox_inches='tight', transparent=True)
    plt.close()
    print(f"✅ Legend saved to {save_path}")


# Animated Cut-View Slices of Spheroid Simulation

This cell defines a function to generate animated GIFs of 2D cut-view slices from 3D spheroid simulation data. For a specified X-plane, it loads voxel and state data across all simulation steps, assigns colors to unique cell states, and renders each slice as an image. The sequence of images is then compiled into an animated GIF, providing a dynamic visualization of how cell states evolve within the selected cross-section over time.

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
import re
from matplotlib.colors import ListedColormap

def make_cut_view_gif(slice_x, batch, condition):
    # --- Settings ---
    print(f"Creating cut view GIF for batch {batch}, condition {condition} at slice x={slice_x}")
    GRID_SIZE = (70, 70, 70)
    input_dir = f"sim_output/{batch}/{condition}/sim_output"
    output_dir = "slice_frames"
    gif_output_path = f"cut_views/cut_view_{batch}_{condition}.gif"

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.dirname(gif_output_path), exist_ok=True)

    # --- Automatically get all available steps ---
    step_files = sorted([
        f for f in os.listdir(input_dir)
        if re.match(r"step_\d+\.jld2", f)
    ])
    step_numbers = [int(re.findall(r"\d+", f)[0]) for f in step_files]

    # --- Build color map for cell states ---
    unique_states = set()
    for fname in step_files:
        with h5py.File(os.path.join(input_dir, fname), "r") as f:
            for k in f["cell_states"]:
                unique_states.add(f["cell_states"][k][()].decode("utf-8"))

    unique_states = sorted(unique_states)
    state_to_idx = {s: i + 1 for i, s in enumerate(unique_states)}  # background = 0

    # Create a custom colormap with black background
    base_cmap = plt.get_cmap("tab10")
    colors = [(0, 0, 0)] + [base_cmap(i / len(unique_states))[:3] for i in range(len(unique_states))]
    custom_cmap = ListedColormap(colors)

    # --- Generate slice images ---
    frame_paths = []

    for step, fname in zip(step_numbers, step_files):
        with h5py.File(os.path.join(input_dir, fname), "r") as f:
            cell_voxels = {
                int(k): np.array([list(t) for t in f["cell_voxels"][k][()]]).astype(int)
                for k in f["cell_voxels"]
            }
            cell_states = {
                int(k): f["cell_states"][k][()].decode("utf-8")
                for k in f["cell_states"]
            }

        slice_image = np.zeros((GRID_SIZE[1], GRID_SIZE[2]), dtype=int)  # Background = 0

        for cid, voxels in cell_voxels.items():
            state = cell_states[cid]
            color_idx = state_to_idx[state]
            for x, y, z in voxels:
                if x == slice_x:
                    slice_image[y, z] = color_idx

        # Plot the slice
        plt.figure(figsize=(5, 5))
        im = plt.imshow(slice_image.T, origin='lower', cmap=custom_cmap, vmin=0, vmax=len(unique_states))
        plt.title(f"Step {step} at x={slice_x}")
        plt.axis("off")

        # Create colorbar skipping background
        cbar = plt.colorbar(im, ticks=range(1, len(unique_states)+1))
        cbar.ax.set_yticklabels(unique_states)

        frame_path = os.path.join(output_dir, f"slice_{step:05d}.png")
        plt.savefig(frame_path, bbox_inches='tight')
        plt.close()
        frame_paths.append(frame_path)

    # --- Create GIF ---
    images = [imageio.imread(f) for f in frame_paths]
    imageio.mimsave(gif_output_path, images, fps=2)

    print(f"✅ Done! GIF saved at: {gif_output_path}")


# 3D Visualization of a Single Spheroid Simulation Step

This cell loads voxel and cell state data from a single simulation step and visualizes the 3D structure of the spheroid using PyVista. Each cell is rendered as a smooth mesh and colored according to its state, with a bounding box, labeled axes, and a legend for clarity. The visualization provides an intuitive overview of the spatial organization and state distribution within the spheroid at the selected time point.

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import marching_cubes
import pyvista as pv

# --- Load your simulation data from file ---
path = "sim_output/batch17/GBM/step_00100.jld2"
with h5py.File(path, "r") as f:
    step = f["step"][()]
    cell_voxels = {
        int(k): np.array([list(t) for t in f["cell_voxels"][k][()]]).astype(int)
        for k in f["cell_voxels"]
    }
    cell_states = {
        int(k): f["cell_states"][k][()].decode("utf-8")
        for k in f["cell_states"]
    }

GRID_SIZE = (70, 70, 70)

# --- Prepare colormap for cell states ---
unique_states = sorted(set(cell_states.values()))
cmap = plt.get_cmap("tab10")
state_colors = {
    state: cmap(i / len(unique_states))[:3]
    for i, state in enumerate(unique_states)
}

# --- Initialize PyVista plotter ---
plotter = pv.Plotter()
plotter.set_background("white")

# --- Helper function to create smooth mesh from voxel positions ---
def voxels_to_mesh(voxels, grid_size):
    volume = np.zeros(grid_size, dtype=np.uint8)
    for x, y, z in voxels:
        if 0 <= x < grid_size[0] and 0 <= y < grid_size[1] and 0 <= z < grid_size[2]:
            volume[x, y, z] = 1
    try:
        verts, faces, _, _ = marching_cubes(volume, level=0.5)
        faces_vtk = np.hstack([[3] + list(face) for face in faces]).astype(np.uint32).reshape(-1, 4)
        mesh = pv.PolyData(verts, faces_vtk)
        return mesh
    except Exception as e:
        print(f"Marching cubes failed for cell: {e}")
        return None

# --- Add each cell as a smooth mesh to the plot ---
for cid, voxels in cell_voxels.items():
    mesh = voxels_to_mesh(voxels, GRID_SIZE)
    if mesh is not None:
        color = state_colors[cell_states[cid]]
        plotter.add_mesh(mesh, color=color, opacity=1.0, show_edges=False)

# --- Add bounding box wireframe ---
bbox = pv.Cube(center=(GRID_SIZE[0]/2, GRID_SIZE[1]/2, GRID_SIZE[2]/2),
               x_length=GRID_SIZE[0], y_length=GRID_SIZE[1], z_length=GRID_SIZE[2])
plotter.add_mesh(bbox, color="black", style="wireframe", opacity=0.1, line_width=1)

# --- Show bounds with only back grid lines, integer ticks automatically ---
plotter.show_bounds(grid='back', location='outer', all_edges=False, ticks='outside',
                    xlabel='X', ylabel='Y', zlabel='Z', font_size=14)

# --- Add legend (small, neat) ---
legend_entries = [(state, tuple(float(c) for c in state_colors[state])) for state in unique_states]
plotter.add_legend(legend_entries, bcolor='white', border=False, size=(0.15, 0.15), loc='upper right')

# --- Camera orientation ---
# X axis forward->back, Y axis right->left (inverted Y), Z vertical up
plotter.view_vector((1, 1, 0.5))

# --- Add title ---
plotter.add_title(f"Cells at step {step}", font_size=20)

# --- Show visualization ---
plotter.show()


# Half-Cut 3D Spheroid Visualization by State Suffix

This cell defines a function to visualize a 3D spheroid simulation with a "half-cut" along a chosen axis, displaying only one side of the structure. Each cell is colored according to the suffix of its state (e.g., proliferative, quiescent, necrotic), providing a clear view of spatial organization and state distribution within the selected half. The function supports customizable axis, side selection, and an optional legend for clarity.

In [6]:
import h5py
import numpy as np
import pyvista as pv
from skimage.measure import marching_cubes

def plot_spheroid_halfcut(
    path,
    grid_size=(70, 70, 70),
    axis="x",              # or "y", "z"
    side="positive",       # "positive" or "negative"
    add_legend=True,
):
    # --- Axis mapping ---
    axis_idx = {"x": 0, "y": 1, "z": 2}[axis]
    center_coord = grid_size[axis_idx] // 2

    # --- Load voxel and state data ---
    with h5py.File(path, "r") as f:
        step = f["step"][()]
        cell_voxels = {
            int(k): np.array([list(t) for t in f["cell_voxels"][k][()]]).astype(int)
            for k in f["cell_voxels"]
        }
        cell_states = {
            int(k): f["cell_states"][k][()].decode("utf-8")
            for k in f["cell_states"]
        }

    # --- Define state suffix colors ---
    def get_state_suffix(state):
        return state.split("_")[-1]

    suffix_colors = {
        "p": (1.0, 0.3, 0.3),  # red
        "q": (0.3, 0.8, 0.3),  # green
        "n": (0.3, 0.3, 1.0),  # blue
    }

    # --- Init PyVista scene ---
    plotter = pv.Plotter()
    plotter.set_background("white")

    # --- Convert voxels to mesh ---
    def voxels_to_mesh(voxels):
        volume = np.zeros(grid_size, dtype=np.uint8)
        for x, y, z in voxels:
            if 0 <= x < grid_size[0] and 0 <= y < grid_size[1] and 0 <= z < grid_size[2]:
                volume[x, y, z] = 1
        try:
            verts, faces, _, _ = marching_cubes(volume, level=0.5)
            faces_vtk = np.hstack([[3] + list(face) for face in faces]).astype(np.uint32).reshape(-1, 4)
            return pv.PolyData(verts, faces_vtk)
        except Exception as e:
            print(f"Marching cubes failed: {e}")
            return None

    # --- Filter and render cells ---
    for cid, voxels in cell_voxels.items():
        # Half-cut filter
        if side == "positive":
            filtered = voxels[voxels[:, axis_idx] >= center_coord]
        else:
            filtered = voxels[voxels[:, axis_idx] < center_coord]

        if len(filtered) == 0:
            continue

        mesh = voxels_to_mesh(filtered)
        if mesh is not None:
            state = cell_states[cid]
            suffix = get_state_suffix(state)
            color = suffix_colors.get(suffix, (0.5, 0.5, 0.5))  # gray fallback
            plotter.add_mesh(mesh, color=color, opacity=1.0, show_edges=False)

    # --- View setup ---
    plotter.view_vector((1, 1, 0.5))
    plotter.show_bounds(
        grid='back', location='outer', all_edges=False,
        xlabel='X', ylabel='Y', zlabel='Z', font_size=14
    )
    #plotter.add_title(f"Spheroid Cut (Step {step})", font_size=18)

    if add_legend:
        legend = [
            ("Proliferative (_p)", suffix_colors["p"]),
            ("Quiescent (_q)", suffix_colors["q"]),
            ("Necrotic (_n)", suffix_colors["n"]),
            ("Other", (0.5, 0.5, 0.5)),
        ]
        plotter.add_legend(legend, bcolor="white", size=(0.2, 0.2), loc="upper right")

    # --- Show it! ---
    plotter.show()
